# Investigating Movie Success

## Introduction
Cinema has become one of the highest profiting industries over the past century. The total box office revenue in North America alone amounted to $11.38 billion in 2016. With the possibility of great success, there is also a large risk of financial failure. This data exploration is motivated by answering the question what makes a movie successful. Specifically do higher budgets mean that a movie will be more profitable? Does the date of release affect the movies profit? Is the box office revenue for the opening weekend of a movie a good predictor of the profit of a movie? Answering these questions could save cinema production companies millions.

## Data Scraped, Downloaded & Cleaned
To begin the data investigation, a collection of movies will be needed to gather data on. To accomplish this, movie names will be scraped from the rotten tomatoes top 100 movies for each genre. There are 17 different genres listed, although some movies are listed in multiple genres, and not every genre contains a full list of 100 movies. In total, after eliminating duplicates, there will be 937 distinct movies collected. The collection of movie names will be biased towards more successful productions, although the large selection will resolve some of the bias introduced by the selection method.

In [1]:
# importing all libraries to be used throughout data exploration
import re, json, requests, seaborn, warnings
warnings.filterwarnings( 'ignore' )
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
from matplotlib import pyplot as plt, rcParams
%matplotlib inline
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from ipyparallel import Client
import time

In [3]:
df = pd.read_csv("Produced_Data/movies_10000.csv",encoding = 'utf8')
vals = ["Position","Const","Created","Modified","Description","Title Type"]
for val in vals:
    del df[val]
df = df[df["Title"]!="The Neon Demon"] # this movie is formatted differently giving a weird error
df1=df[:2000]
df2=df[2000:4000]
df3=df[4000:6000]
df4=df[6000:8000]
df5=df[8000:]
dfs = [df1,df2,df3,df4,df5]

In [4]:
def text_message(text,number):
    browser = webdriver.Chrome()
    url = "http://www.txtdrop.com/"
    browser.get( url )
    email = browser.find_element_by_id("emailfrom")
    email.send_keys("brookemosby@hotmail.com")
    first_3 = browser.find_element_by_id("npa")
    first_3.send_keys(number[:3])
    second_3 = browser.find_element_by_id("exchange")
    second_3.send_keys(number[3:6])
    last_4 = browser.find_element_by_id("number")
    last_4.send_keys(number[6:])
    message = browser.find_element_by_name("body")
    message.send_keys(text)
    browser.find_element_by_name("submit").click()
    browser.close()

In [60]:
#Gathering data from IMDB
browser = webdriver.Chrome()
def gather_data(row):
    #Function accepts a pd.Series representing a pd.DataFrame row, then 
    #changes the budget and opening weekend monetary amount to USD, and
    #returns the new updated pd.Series with the correct amounts.
    url = row[ 1 ]
    browser.get( url )
    soup = BeautifulSoup( browser.page_source, "html.parser" )
    time.sleep(10*np.random.random())
    # All information needed is contained in h4 tabs
    tab = soup.find_all( name = "h4" )
    B, O, G = False, False, False
    for j in tab:
        if j.text == "Budget:":
            row["Budget"] = ( " " ).join( j.next_sibling.split() )
            B = True
        if j.text == "Opening Weekend:":
            row["Opening Weekend"] = ( " " ).join( j.next_sibling.split( ) )
            O = True
        if "Gross" in j.text :
            row["Gross"] = ( " " ).join( j.next_sibling.split( ) ) 
            G = True
    span = soup.find_all( itemprop = "awards" )
    text = ""
    for j in span:
        text += " " + j.text
    text = text.split()
    if "nominations." in text  or "nomination." in text:
        row["Other Nominations"] = int(text[-2])
    else: row["Other Nominations"] = 0
    if "wins." in text:
        row["Other Wins"] = int(text[text.index("wins.")-1])
    elif "wins" in text:
        row["Other Wins"] = int(text[text.index("wins")-1])    
    elif "win." in text:
        row["Other Wins"] = int(text[text.index("win.")-1])
    elif "win" in text:
        row["Other Wins"] = int(text[text.index("win")-1])
    else: row["Other Wins"] = 0
    if "Oscar." in text or "Oscars." in text:
        if "Nominated" in text:
            row["Oscar Nominations"] = int(text[2])
            row["Oscar Wins"] = 0
        else:
            row["Oscar Wins"] = int(text[1])
            row["Oscar Nominations"] = 0
    else:
        row["Oscar Wins"] = 0
        row["Oscar Nominations"] = 0

    # Inserting NaNs for missing values
    if not B: row["Budget"] = np.nan
    if not O: row["Opening Weekend"] = np.nan
    if not G: row["Gross"] = np.nan
    try: row["Meta Score"] = int(soup.find( "div",{"class":"metacriticScore"} ).text)
    except: row["Meta Score"] = np.nan
    try: row["Content Rating"] = soup.find( itemprop = "contentRating" )["content"]
    except:
        try: row["Content Rating"] = soup.find( itemprop = "contentRating" )["certificate"]
        except: row["Content Rating"] = "NR"

    actors = soup.find_all( "span",itemprop = "name" )
    actors =[ actor.text for actor in actors][:10]
    for i in range(10):
        try: row["Actor_"+str(i)] = actors[i]
        except: row["Actor_"+str(i)] = "NA"
    return row
df1 = df4.apply(gather_data,axis=1)
browser.close()
text_message("Finished","6787995970")

In [61]:
df1.to_csv("Result_Data/third_2000_uncleaned.csv")

In [101]:
df1 = pd.read_csv("Result_Data/third_2000_uncleaned.csv")#accidentally overwrote third, need to redo engineering
df1.sample(5)

,Unnamed: 0,Actor_0,Actor_1,Actor_2,Actor_3,Actor_4,Actor_5,Actor_6,Actor_7,Actor_8,...,Opening Weekend,Oscar Nominations,Oscar Wins,Other Nominations,Other Wins,Release Date,Runtime (mins),Title,URL,Year
1124,5125,Stephen Daldry,Christian Duurvoort,Andy Mulligan,Felipe Braga,Rickson Tevez,Gabriel Weinstein,Martin Sheen,Rickson Tevez,Wagner Moura,...,NaN,0,0,4,2,2014-10-07,114.0,Trash,http://www.imdb.com/title/tt1921149/,2014
1318,5319,Daniel Ribeiro,Daniel Ribeiro,Ghilherme Lobo,Fabio Audi,Tess Amorim,Ghilherme Lobo,Fabio Audi,Tess Amorim,Lúcia Romano,...,"BRL 456,110 (Brazil),",0,0,14,23,2014-02-10,96.0,Hoje Eu Quero Voltar Sozinho,http://www.imdb.com/title/tt1702014/,2014
1888,5889,Matthew Weiner,Matthew Weiner,Owen Wilson,Zach Galifianakis,Amy Poehler,Owen Wilson,Zach Galifianakis,Jay Gates,Jason Davis,...,NaN,0,0,0,0,2013-09-07,114.0,Are You Here,http://www.imdb.com/title/tt1545754/,2013
802,4803,Daniel Attias,Stephen King,Stephen King,Gary Busey,Everett McGill,Corey Haim,Gary Busey,Everett McGill,Corey Haim,...,NaN,0,0,1,0,1985-10-11,95.0,Silver Bullet,http://www.imdb.com/title/tt0090021/,1985
166,4167,Fredrik Bond,Matt Drake,Shia LaBeouf,Evan Rachel Wood,Mads Mikkelsen,Shia LaBeouf,Evan Rachel Wood,Mads Mikkelsen,Til Schweiger,...,NaN,0,0,1,0,2013-01-21,103.0,The Necessary Death of Charlie Countryman,http://www.imdb.com/title/tt1196948/,2013


In [88]:
df1.columns

Index(['Unnamed: 0', 'Actor_0', 'Actor_1', 'Actor_2', 'Actor_3', 'Actor_4',
       'Actor_5', 'Actor_6', 'Actor_7', 'Actor_8', 'Actor_9', 'Budget',
       'Content Rating', 'Directors', 'Genres', 'Gross', 'IMDb Rating',
       'Meta Score', 'Num Votes', 'Opening Weekend', 'Oscar Nominations',
       'Oscar Wins', 'Other Nominations', 'Other Wins', 'Release Date',
       'Runtime (mins)', 'Title', 'URL', 'Year'],
      dtype='object')

In [102]:
genres = set()
for g_l in df1['Genres']:
    for genre in g_l.split(','):
        genres.add(genre)
genres = list(genres)
def genre_bool(row):
    for genre in genres:
        if genre in row['Genres']:
            row["Genre: " +genre]=1
        else:
            row["Genre: " +genre]=0
    return row
df1 = df1.apply(genre_bool, axis = 1)
del df1['Genres']

ratings = set()
for r in df1['Content Rating']:
    ratings.add(r)
ratings = list(ratings)
def rating_bool(row):
    for rating in ratings:
        if rating == row['Content Rating']:
            row["Content Rating: "+rating]=1
        else:
            row["Content Rating: "+rating]=0
    return row

df1 = df1.apply(rating_bool, axis = 1)
del df1['Content Rating']
df1.head()

,Unnamed: 0,Actor_0,Actor_1,Actor_2,Actor_3,Actor_4,Actor_5,Actor_6,Actor_7,Actor_8,...,Content Rating: GP,Content Rating: PG,Content Rating: PG-13,Content Rating: G,Content Rating: NR,Content Rating: UNRATED,Content Rating: APPROVED,Content Rating: M/PG,Content Rating: NOT RATED,Content Rating: R
0,4001,Joe Dante,Gary Brandner,John Sayles,Dee Wallace,Patrick Macnee,Dennis Dugan,Dee Wallace,Patrick Macnee,Dennis Dugan,...,0,0,0,0,0,0,0,0,0,1
1,4002,Woody Allen,Woody Allen,Mickey Rose,Woody Allen,Janet Margolin,Marcel Hillaire,Woody Allen,Janet Margolin,Marcel Hillaire,...,0,0,0,0,0,0,0,0,0,0
2,4003,Pedro Almodóvar,Pedro Almodóvar,Jorge Guerricaechevarría,Liberto Rabal,Francesca Neri,Javier Bardem,Javier Bardem,Francesca Neri,Liberto Rabal,...,0,0,0,0,0,0,0,0,0,1
3,4004,Steve Shill,David Loughery,Beyoncé,Idris Elba,Ali Larter,Idris Elba,Beyoncé,Ali Larter,Jerry O'Connell,...,0,0,1,0,0,0,0,0,0,0
4,4005,J. Lee Thompson,Pierre Boulle,Paul Dehn,Roddy McDowall,Claude Akins,Natalie Trundy,Roddy McDowall,Claude Akins,Natalie Trundy,...,0,0,0,1,0,0,0,0,0,0


In [103]:
del df1["Unnamed: 0"]
del df1["URL"]

In [104]:
columns = df1.columns
for i, col in enumerate(df1.columns):
    print(i, col)

0 Actor_0
1 Actor_1
2 Actor_2
3 Actor_3
4 Actor_4
5 Actor_5
6 Actor_6
7 Actor_7
8 Actor_8
9 Actor_9
10 Budget
11 Directors
12 Gross
13 IMDb Rating
14 Meta Score
15 Num Votes
16 Opening Weekend
17 Oscar Nominations
18 Oscar Wins
19 Other Nominations
20 Other Wins
21 Release Date
22 Runtime (mins)
23 Title
24 Year
25 Genre:  Comedy
26 Genre: Fantasy
27 Genre: Film-Noir
28 Genre: War
29 Genre: Musical
30 Genre:  Sport
31 Genre: Biography
32 Genre: Action
33 Genre:  Fantasy
34 Genre:  Biography
35 Genre: Mystery
36 Genre:  Musical
37 Genre:  Romance
38 Genre: Thriller
39 Genre:  Film-Noir
40 Genre:  History
41 Genre: Western
42 Genre: Drama
43 Genre: Sci-Fi
44 Genre:  Horror
45 Genre: Romance
46 Genre: Adventure
47 Genre:  Family
48 Genre:  Sci-Fi
49 Genre: Animation
50 Genre:  Music
51 Genre: Music
52 Genre:  Mystery
53 Genre:  Thriller
54 Genre: Comedy
55 Genre:  Crime
56 Genre: Horror
57 Genre:  Drama
58 Genre:  War
59 Genre:  Western
60 Genre:  Adventure
61 Genre: Family
62 Genre:  Act

### Gathering Monetary Information on Each Movie
After accumulating movie names, motion picture content rating, and the rotten tomatoes score, the dataset is still missing information on the movie budget, opening weekend revenue, and gross income, which will provide the numerical data points necessary to understand the factors that make a movie successful. The data points will be collected from IMDB, which is a reputable source for information, according to their website, 

>"we [IMDB] actively gather information from and verify items with studios and 
filmmakers".

While collecting the data points, it will be convient and valuable to find the release date of the movie and add to the dataset.

### Uncleaned Dataset

## Cleaning Data
After gathering each data point, the data set is complete, although the information is not clean or uniform. The first step to clean up the names and release dates, will be to remove any phrases in parenthesis across the rows of the DataFrame. Following the removal of parenthesis, will be the removal of all commas across each column in the DataFrame. Removing commas will make it easier to convert monetary amounts to ints. Next each date in the Release column will be changed to a pandas date object, which will simplify any calculations that rely on the release date of the movie. Finally each monetary amount will be converted into an int, while creating place holder columns for the currency type of the budget and opening weekend revenue. The gross income column of the movie DataFrame is listed in USD solely, so this value will not need a place holder column for it. The dataset is now cleaned, although the information is not given in uniform units, which will need to be addressed.

In [92]:
def remove_parenthesis( row ):
    """
    Function accepts a pd.Series describing a row of a DataFrame. For each 
    value in the row the string before the first opening parenthesis becomes 
    the new value. The new pd.Series of the row is returned, without any 
    phrases in parenthesis.
    """
    row = [ str( value ).split( "(" )[ 0 ] for value in row ]
    return pd.Series( row )
def remove_comma( row ):
    """
    Function accepts a pd.Series describing a row of a DataFrame. For each 
    value in the row the string is split by commas, then joined without 
    them. The new pd.Series of the row is returned, without commas.
    """
    row = [ ( "" ).join( str( value ).split( "," ) ) for value in row ]
    return pd.Series( row )
def _type( currency ):
    """
    Function accepts a value representing a monetary amount. If the value 
    is a string "nan", then np.nan is returned. If the monetary amount split
    by spaces has length greater than one, then this implies that there is a 
    string, such as AUD, before the monetary amount, this currency indicator 
    will be returned. If the monetary amount split by spaces only consist of 
    one item, that implies a symbol, such as $ is before the monetary amount. 
    This currency indicator will be returned.
    """
    if currency == np.nan:
        return np.nan
    currency_indicator = currency.split()
    if len( currency_indicator ) > 1:
        return currency_indicator[ 0 ]
    else:
        return currency[ 0 ]
def _int( currency ):
    """
    Function accepts a value representing a monetary amount. If the value is 
    a string "nan", then np.nan is returned. If the monetary amount split by
    spaces has length greater than one, then this implies that there is a 
    string, such as AUD, before the monetary amount, the monetary amount 
    after the currency indicator will be returned as an int. If the monetary 
    amount split by spaces only consist of one item, that implies a symbol,
    such as $ is before the monetary amount. The monetary amount after the 
    currency indicator will be returned as an int.
    """
    if currency == "nan":
        return np.nan
    currency_amount = currency.split()
    if len( currency_amount ) > 1:
        return int( currency_amount[ 1 ] )
    else:
        return int( currency[ 1: ] )
    
df1      = df1.apply( remove_parenthesis, axis = 1 )
df1      = df1.apply( remove_comma,       axis = 1 )
df1[21] = df1[21].apply( lambda x: pd.to_datetime( str( x ) ))
df1[24] = df1[24].apply( lambda x: pd.to_datetime( str( x ) ) )
df1[78] = df1[10].apply( _type )
df1[8] = df1[16].apply( _type )
df1[10] = df1[10].apply( _int  ) #
df1[12] = df1[12].apply( _int  ) #
df1[16] = df1[16].apply( _int  ) #
df1.head()

,0,1,2,3,4,5,6,7,8,9,...,72,73,74,75,76,77,78,79,80,81
0,Ron Howard,Lowell Ganz,Babaloo Mandel,Henry Winkler,Michael Keaton,Shelley Long,Henry Winkler,Michael Keaton,Shelley Long,Gina Hecht,...,0,0,0,0,0,0,0,1,n,n
1,Mark Pellington,Albert Torres,Luke Wilson,Radha Mitchell,George Lopez,Luke Wilson,Radha Mitchell,Adriana Barraza,George Lopez,Cheryl Hines,...,0,0,0,0,0,0,0,0,n,n
2,Marco Kreuzpaintner,Thomas Bahmann,Marco Kreuzpaintner,Robert Stadlober,Kostja Ullmann,Miriam Morgenstern,Robert Stadlober,Kostja Ullmann,Jürgen Tonkel,Miriam Morgenstern,...,0,0,0,0,0,0,0,1,€,n
3,Roman Polanski,Gérard Brach,Roman Polanski,Nastassja Kinski,Peter Firth,Leigh Lawson,John Collin,Tony Church,Nastassja Kinski,Brigid Erin Bates,...,0,0,0,0,0,0,0,0,$,n
4,Nacho Cerdà,Karim Hussain,Nacho Cerdà,Anastasia Hille,Karel Roden,Valentin Goshev,Anastasia Hille,Karel Roden,Valentin Ganev,Paraskeva Djukelova,...,0,0,0,0,0,0,0,1,€,n


### Converting Currency to USD
The next step in unifying the currency units is to convert all monetary amounts into USD. Converting each column to USD will make comparisons valid, and is accomplished by using a dataset which contains the exchange rates for 26 different currencies to USD, for dates throughout the past century. For currencies not listed in USD, the release date of the movie will be used to find the accurate exchange rate to USD, after which the currency will be converted to USD. Once the conversations are calculated, the place holder columns describing the original currency are dropped, resulting in a clean and uniform dataset.

In [93]:
e_r_columns = [ "Dates",   "€",   "£",  "BRL", "CNY", "DKK", "INR", "JPY", 
                  "KRW", "MYR", "MXN",  "NOK", "SEK", "ZAR", "SGD", "CHF", 
                  "TWD", "THB", "VEB", "NBDI", "NMC", "NOI", "AUD", "NZD", 
                  "CAD", "HKD", "LKR" ]
exchange_rates = pd.read_csv( "Produced_Data/exchange_rates.csv", 
                             skiprows = [ 0, 1, 2, 4, 5, 6 ] )
exchange_rates.columns = e_r_columns
exchange_rates[ "Dates" ] = exchange_rates[ "Dates" ].apply( 
    lambda x: pd.to_datetime( str( x ) ) )

def change_to_USD(row):
    """
    Function accepts a pd.Series representing a pd.DataFrame row, then 
    changes the budget and opening weekend monetary amount to USD, and
    returns the new updated pd.Series with the correct amounts.
    """
    date = row[21]
    foreign_budget = row[10]
    foreign_open   = row[16]
    indicator_budget = row[78]
    indicator_open   = row[79]
    
    if indicator_budget != "$" and indicator_budget != "nan":
        try:
            cur_rate = exchange_rates[ indicator_budget ][ 
                exchange_rates[ "Dates" ] == date ].values[ 0 ]
            row[10] = int( foreign_budget ) / float( cur_rate )
        except:
            row[10] = np.nan
    if indicator_open   != "$" and indicator_open   != "nan":
        try:
            cur_rate = exchange_rates[ indicator_open   ][ 
                exchange_rates[ "Dates" ] == date ].values[ 0 ]
            row[16] = int( foreign_open )  / float( cur_rate )
        except:
            row[16] = np.nan
    return row

df1 = df1.apply( change_to_USD, axis = 1 )

# Delete currency indicator columns
del df1[80], df1[81]
df1.columns = columns
df1.sample( 3 )

,Actor_0,Actor_1,Actor_2,Actor_3,Actor_4,Actor_5,Actor_6,Actor_7,Actor_8,Actor_9,...,Content Rating: PG,Content Rating: PG-13,Content Rating: NR,Content Rating: G,Content Rating: APPROVED,Content Rating: UNRATED,Content Rating: TV-13,Content Rating: NOT RATED,Content Rating: TV-G,Content Rating: R
1098,Nnegest Likké,Nnegest Likké,Mo'Nique,Kendra C. Johnson,Jimmy Jean-Louis,Mo'Nique,Jimmy Jean-Louis,Godfrey,Kendra C. Johnson,Joyful Drake,...,0,1,0,0,0,0,0,0,0,0
46,Hector Babenco,Manuel Puig,Leonard Schrader,William Hurt,Raul Julia,Sônia Braga,William Hurt,Raul Julia,Sônia Braga,José Lewgoy,...,0,0,0,0,0,0,0,0,0,1
203,Federico Fellini,Federico Fellini,Ennio Flaiano,Alberto Sordi,Franco Fabrizi,Franco Interlenghi,Franco Interlenghi,Alberto Sordi,Franco Fabrizi,Leopoldo Trieste,...,0,0,0,0,0,0,0,1,0,0


In [94]:
df1.head()

,Actor_0,Actor_1,Actor_2,Actor_3,Actor_4,Actor_5,Actor_6,Actor_7,Actor_8,Actor_9,...,Content Rating: PG,Content Rating: PG-13,Content Rating: NR,Content Rating: G,Content Rating: APPROVED,Content Rating: UNRATED,Content Rating: TV-13,Content Rating: NOT RATED,Content Rating: TV-G,Content Rating: R
0,Ron Howard,Lowell Ganz,Babaloo Mandel,Henry Winkler,Michael Keaton,Shelley Long,Henry Winkler,Michael Keaton,Shelley Long,Gina Hecht,...,0,0,0,0,0,0,0,0,0,1
1,Mark Pellington,Albert Torres,Luke Wilson,Radha Mitchell,George Lopez,Luke Wilson,Radha Mitchell,Adriana Barraza,George Lopez,Cheryl Hines,...,1,0,0,0,0,0,0,0,0,0
2,Marco Kreuzpaintner,Thomas Bahmann,Marco Kreuzpaintner,Robert Stadlober,Kostja Ullmann,Miriam Morgenstern,Robert Stadlober,Kostja Ullmann,Jürgen Tonkel,Miriam Morgenstern,...,0,0,0,0,0,0,0,0,0,1
3,Roman Polanski,Gérard Brach,Roman Polanski,Nastassja Kinski,Peter Firth,Leigh Lawson,John Collin,Tony Church,Nastassja Kinski,Brigid Erin Bates,...,1,0,0,0,0,0,0,0,0,0
4,Nacho Cerdà,Karim Hussain,Nacho Cerdà,Anastasia Hille,Karel Roden,Valentin Goshev,Anastasia Hille,Karel Roden,Valentin Ganev,Paraskeva Djukelova,...,0,0,0,0,0,0,0,0,0,1


In [95]:
df1.to_csv( "Result_Data/fourth_2000_cleaned.csv" )

## Feature Engineering
Some adjustments to the dataset will be needed in order to better understand the data. First it will be beneficial to create a column corresponding to the decades of the movies' release date. A column that identifies the profit of the movie will be crucial in determining the financial success of the movie. To create a Profit column, the budget of the movie will be subtracted from the gross income. Finally, there is one remaining issue to be addressed. Because the movies in the dataset span the past century the monetary amounts are not adjusted for inflation, making them difficult to compare directly. To resolve the disagreement between the monetary amounts, a dataset containing the CPI for each year from 1914 will used to adjust for inflation. The CPI, Consumer Price Index, describes the amount of purchasing power the average consumer has. To account for inflation, the current CPI is divided by the CPI of the year when the movie was released to obtain a ratio of purchasing power. The monetary amounts are multiplied by this ratio, resulting in a new monetary amount corresponding to the approximate amount with today's purchasing power. The budget, opening weekend revenue, gross income, and profit of each movie will be adjusted for inflation, and placed into new columns. With the adjusted inflation columns, relationships between variables may become more apparent.

In [98]:
df1=df1.drop(1281)

In [99]:
# Calculating the decade for the release date
df1[ "Decade" ] = df1[ "Release Date" ].apply( lambda x: ( ( x.year % 1900 ) // 10 ) * 10 + 1900  ) 
df1[ "Profit" ] = df1[ "Gross" ] - df1[ "Budget" ]

# Adjusting for inflation
inflation_rates = pd.read_csv( "Produced_Data/inflation_rates.csv", index_col = "Year")
current_CPI = float( inflation_rates[ "November" ].iloc[-1] )
def change_to_current(row):
    """
    Function accepts pd.Series describing a row in pd.DataFrame
    Then creates new columns adjusting for inflation by multiplying by CPI ratio
    Function returns new pd.Series describing row in pd.DataFrame
    """
    year_release  = row[ "Release Date" ].year
    month_release = row[ "Release Date" ].month - 1
    value_CPI = float( inflation_rates.loc[ year_release ][ 12 ] )
    if value_CPI == 0 :
        row[ "Budget_Adjusted" ], row[ "Gross_Adjusted"  ] = np.nan, np.nan
        row[ "Open_Adjusted"   ], row[ "Profit_Adjusted" ] = np.nan, np.nan
    else:
        row[ "Budget_Adjusted" ] = row[ "Budget" ] * current_CPI / value_CPI
        row[ "Gross_Adjusted"  ] = row[ "Gross"  ] * current_CPI / value_CPI
        row[ "Open_Adjusted"   ] = row[ "Opening Weekend"   ] * current_CPI / value_CPI
        row[ "Profit_Adjusted" ] = row[ "Profit" ] * current_CPI / value_CPI
    return row
df1 = df1.apply( change_to_current, axis = 1 )
df1.head(3)

,Actor_0,Actor_1,Actor_2,Actor_3,Actor_4,Actor_5,Actor_6,Actor_7,Actor_8,Actor_9,...,Content Rating: TV-13,Content Rating: NOT RATED,Content Rating: TV-G,Content Rating: R,Decade,Profit,Budget_Adjusted,Gross_Adjusted,Open_Adjusted,Profit_Adjusted
0,Ron Howard,Lowell Ganz,Babaloo Mandel,Henry Winkler,Michael Keaton,Shelley Long,Henry Winkler,Michael Keaton,Shelley Long,Gina Hecht,...,0,0,0,1,1980,NaN,NaN,6.032527e+07,NaN,NaN
1,Mark Pellington,Albert Torres,Luke Wilson,Radha Mitchell,George Lopez,Luke Wilson,Radha Mitchell,Adriana Barraza,George Lopez,Cheryl Hines,...,0,0,0,0,2000,NaN,NaN,2.104127e+06,NaN,NaN
2,Marco Kreuzpaintner,Thomas Bahmann,Marco Kreuzpaintner,Robert Stadlober,Kostja Ullmann,Miriam Morgenstern,Robert Stadlober,Kostja Ullmann,Jürgen Tonkel,Miriam Morgenstern,...,0,0,0,1,2000,NaN,NaN,1.240845e+05,NaN,NaN


In [100]:
df1.to_csv( "Result_Data/fourth_2000_engineered.csv" )

## Visualizing Data
The next step to understanding relationships between different variables is visualizing the data. The first step is to investigate how the data has changed over the past century, and compare the change between variables, which may give some insight into relationships between variables.

In [ ]:
plt.figure( figsize = ( 17, 8 ) )
ax4      = plt.subplot( 414 )
ax1, ax2 = plt.subplot( 411, sharex = ax4 ), plt.subplot( 412, sharex = ax4 )
ax3      = plt.subplot( 413, sharex = ax4 )
df.plot( y = "Gross",  x = "Release", ax = ax1, title = "Gross by Release Date" )
df.plot( y = "Budget", x = "Release", ax = ax2, title = "Budget by Release Date" )
df.plot( y = "Open",   x = "Release", ax = ax3, title = "Opening Weekend by Release Date" )
df.plot( y = "Profit", x = "Release", ax = ax4, title = "Profit by Release Date" )
plt.show()

After observing the graph above, it is apparrent there has been an exponential increase in movie budgets, opening weekend revenues, gross income and profit. It is still unclear how much of this increase is due to inflation over time. Investigating how inflation has changed these four variables is the next rational step.

In [ ]:
plt.figure( figsize = ( 17, 8 ) )
ax4      = plt.subplot( 414 )
ax1, ax2 = plt.subplot( 411, sharex = ax4 ), plt.subplot( 412, sharex = ax4 )
ax3      = plt.subplot( 413, sharex = ax4 )
df.plot( y = "Gross_Adjusted",  x = "Release", ax = ax1, 
        title = "Inflation Adjusted Gross by Release Date" )
df.plot( y = "Budget_Adjusted", x = "Release", ax = ax2, 
        title = "Inflation Adjusted Budget by Release Date" )
df.plot( y = "Open_Adjusted",   x = "Release", ax = ax3, 
        title = "Inflation Adjusted Opening Weekend by Release Date" )
df.plot( y = "Profit_Adjusted", x = "Release", ax = ax4, 
        title = "Inflation Adjusted Profit by Release Date" )
plt.show()

Looking at the variables in the graphs above, it looks as though the the gross income and profit have decreased over time, while the opening weekend revenue and budget have increased over time. It is important to note that there are significantly more movies released in later decades, as demonstrated in the histogram below. 

In [ ]:
df["Decade"].hist( figsize = ( 17, 2 ) )
plt.show()

It is interesting to note that the movie with the highest adjusted budget was Titanic, the movie with the highest opening weekend revenue was Harry Potter and the Deathly Hallows - Part 2 and the movie with both the highest gross income and profit was Gone with the Wind. After looking at how the monetary variables are distributed over the past century, it seems reasonable to investigate how the variables are correlated.

In [ ]:
plt.figure( figsize = ( 17,3 ) )
seaborn.heatmap( df[["Decade", "Budget_Adjusted", "Gross_Adjusted", "Open_Adjusted", 
                     "Profit_Adjusted"]].corr(), annot = True, fmt = ".001f" )
plt.show()

The heatmap above gives the strengths of the correlations coefficients, on a scale of -1.0 to 1.0. 1.0 is perfectly positively correlated, and -1.0 is perfectly negatively correlated, meaning each variable will positively correlate to itself perfectly. The heatmap gives some interesting relationships between variables. There were no strong correlations between the unadjusted variables, and thus were excluded from the heatmap above. The first relationship worth notice, is that the later in the century a movie was produced, the worse the adjusted profit and adjusted gross income of the movie would be. A second relationship that needs investigation is the perfect positive correlation between the adjusted gross and the adjusted profit. Since the adjusted profit is calculated from the adjusted gross income, it is reasonable that they would be strongly positively correlated, although extremely unlikely that this would be a perfect relationship. The correlation coefficient for the relationship between the adjusted gross income and the adjusted profit is 0.978601533586, which is strongly positive, although not perfect, as the rounded number in the heatmap would suggest. It is interesting to note that from this heatmap, it seems as though the adjusted budget has little correlation to the adjusted profit. The relationships between the adjusted profit for a movie, and the adjusted budget, adjusted gross income, and adjusted opening weekend revenue still need to be investigated. There may be some intuition to gain from looking at the linear regression for each relationship, which would be the next step in the data exploration.

In [ ]:
plt.figure( figsize = ( 17,3 ) )
ax1 = plt.subplot( 131 )
ax2, ax3 = plt.subplot( 132, sharey = ax1 ), plt.subplot( 133, sharey = ax1 )
seaborn.regplot( x = "Budget_Adjusted", y ="Profit_Adjusted", ax = ax1, data = df ) 
seaborn.regplot( x = "Open_Adjusted",   y ="Profit_Adjusted", ax = ax2, data = df ) 
seaborn.regplot( x = "Gross_Adjusted",  y ="Profit_Adjusted", ax = ax3, data = df ) 
plt.show()

From the graphs above, it is clear that the gross income is the best estimator for the profit, mainly because profit is calculated from the gross income. However the gross income is only useful to estimate the profit, if the variable for the budget is not available. The second best variable for predicting the profit is the opening weekend revenue. The opening weekend revenue may be a more useful predictor for profit than the gross income, because it can give insight into what the profit of a movie may be before it leaves theaters. It is interesting that there is almost no correlation between the budget of a movie, and the profit the movie makes.

In [ ]:
df[["Budget_Adjusted","Open_Adjusted","Gross_Adjusted","Profit_Adjusted"]].plot( 
    kind = "kde", colormap='Set1', figsize = (17,5) )
plt.xlim(-.5e8,1e8)
plt.show()

Looking at the distributions above we can see that there is a large variance between the adjusted gross income and the adjusted profit, with almost not peak. Whereas, the adjusted budget and the adjusted open weekend revenue are less varied distributions. The adjusted budget has little or no correlation to the adjusted profit, which suggests that the regardless of the budget of a movie, there is still a high risk to lose alot of money.

## Conclusion
Unfortunately there does not seem to be a large correlation between the budget of a movie, and the profit of the production. This suggests that spending large amounts of money on a movie production does not necessarily pay off.  After looking into all the dataset variables, there was little information to be gained from the motion picture content rating, the rotten tomatoes score, and the unadjusted monetary amounts. One of the most useful predictors for the profit of a movie seems to be the opening weekend revenue, which is intuitive. A larger dataset of movies could present more correlations, due to time constraints this dataset was limited to ~1000 movie titles.  More information that could be used as predictors for the profit of a movie would be money and time spent into marketing the movie, and the amount of different trailers released. These variables may play a larger role and factor into what makes a movie more profitable and successfull. Also, collecting information on DVD sales once the movie left the theater could give more insight into just how profitable, or successful the movie was.